<a href="https://colab.research.google.com/github/deltorobarba/sciences/blob/master/cern.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **CERN Quantum Computing in High-Energy Physics 2022**

***Event***: [*International Conference on Quantum Technologies for High-Energy Physics (QT4HEP22)*](https://indico.cern.ch/event/1190278/)

**Annalisa Pawlosky, PhD**, *Research Scientist at Google*

[Google Scholar](#) $\cdot$ [LinkedIn](https://www.linkedin.com/in/annalisa-pawlosky-bb7579178/)

**Alexander Del Toro Barba, PhD**, *Quantum Computing Practice Lead at Google Cloud*

[Google Scholar](https://scholar.google.com/citations?hl=en&user=fddyK-wAAAAJ) $\cdot$ [LinkedIn](https://www.linkedin.com/in/deltorobarba/)


###### *Libraries*

In [ ]:
!pip install tensorflow==2.7.0 -q
!pip install tensorflow-quantum==0.7.2 -q

import importlib, pkg_resources
importlib.reload(pkg_resources)

import tensorflow as tf
import tensorflow_quantum as tfq

import cirq
import sympy
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
from cirq.contrib.svg import SVGCircuit

print('Cirq version:', cirq.__version__)
print('TensorFlow version:', tf.__version__)
print('TensorFlow Quantum version:', tfq.__version__)

#### **Part 1: Cirq**

https://quantumai.google/cirq

In [ ]:
# Pick a qubit.
qubit = cirq.GridQubit(0, 0)

# Create a circuit
circuit = cirq.Circuit(
    cirq.X(qubit)**0.5,  # Square root of NOT.
    cirq.measure(qubit, key='m')  # Measurement.
)
print("Circuit:")
print(circuit)

# Simulate the circuit several times.
simulator = cirq.Simulator()
result = simulator.run(circuit, repetitions=20)
print("Results:")
print(result)

Circuit:
(0, 0): ───X^0.5───M('m')───
Results:
m=10011100110010100110


#### **Part 2: TensorFlow Quantum**

https://www.tensorflow.org/quantum

#### **Part 3: QuantumVM**

[Quantum-Virtual-Machine](https://quantumai.google/quantum-virtual-machine)

* [Documentation here](https://quantumai.google/cirq/simulate/quantum_virtual_machine)

* [Launch QVM learning version in Colab](https://colab.sandbox.google.com/github/quantumlib/Cirq/blob/master/docs/simulate/qvm_basic_example.ipynb)